## Preguntas a responder:
- Cuál es el cliente típico?
- Qué diferencia un comprador de alguien curioso? 
- Cómo se distribuyen las propiedades?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import geopy

In [ ]:
# --- 1. Definir los colores de la marca Colibid ---
# Ajusta estos códigos hexadecimales si tienes los colores exactos de la guía de estilo de Colibid
COLORES_COLIBID = {
    'primary_blue_green': '#03897D', # Turquesa/Azul verdoso (similar al del PDF)
    'secondary': '#80BEAB',
    'secondary_orange': '#FF8C00',  # Naranja vibrante (similar al de los elementos naranjas del PDF)
    'text_dark': '#333333',         # Gris oscuro para el texto
    'background_light': '#FFFFFF'   # Blanco puro para fondos
}

# --- 2. Configurar el estilo de Matplotlib y Seaborn ---
# Utilizar un estilo de Seaborn que sea limpio y luego aplicar nuestras personalizaciones
sns.set_theme(style="whitegrid", palette="deep") # "whitegrid" para el fondo blanco con cuadrículas
plt.rcParams['font.family'] = 'Segoe UI' # Fuente limpia, puedes probar 'Arial', 'Verdana'
plt.rcParams['axes.titlecolor'] = COLORES_COLIBID['text_dark']
plt.rcParams['axes.labelcolor'] = COLORES_COLIBID['text_dark']
plt.rcParams['xtick.color'] = COLORES_COLIBID['text_dark']
plt.rcParams['ytick.color'] = COLORES_COLIBID['text_dark']
plt.rcParams['figure.facecolor'] = COLORES_COLIBID['background_light']
plt.rcParams['axes.facecolor'] = COLORES_COLIBID['background_light']
plt.rcParams['grid.color'] = '#CCCCCC' # Un gris claro para las líneas de la cuadrícula
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.alpha'] = 0.7

# Crear una paleta de Seaborn personalizada con los colores de Colibid
# Esto es útil para cuando seaborn asigna colores automáticamente
colibid_palette = [COLORES_COLIBID['primary_blue_green'], COLORES_COLIBID['secondary_orange'],
                   '#7F8C8D', '#9B59B6', '#34495E', '#E74C3C'] # Añade más colores si tienes más categorías

# Puedes establecer esta paleta como la predeterminada de Seaborn si quieres
# sns.set_palette(colibid_palette)

print("Estilo de Colibid configurado para Matplotlib y Seaborn.")

In [ ]:
df_dataset_v2 = pd.read_csv('data/dataset_v2.csv')
df_ofertas_aceptadas = pd.read_csv('data/ofertas_aceptadas.csv')
df_ofertas = pd.read_csv('data/ofertas.csv')
df_registros = pd.read_csv('data/registros.csv')
df_solicitudes = pd.read_csv('data/solicitudes_de_hipoteca.csv')

### Exploramos cada dataset para determinar qué queremos estudiar

In [ ]:
df_registros.head(2)

In [ ]:
df_registros.iloc[2,:]

In [ ]:
df_solicitudes.head(2)

In [ ]:
df_solicitudes.iloc[0,:]

In [ ]:
json.loads(df_solicitudes.iloc[50,2])['user']#['data']#['address']

In [ ]:
json.loads(df_solicitudes.iloc[1,2])['property']

In [ ]:
df_ofertas.head(2)

In [ ]:
json.loads(df_ofertas.iloc[8, 6])['id']
json.loads(df_ofertas.iloc[8, 6])['name']

In [ ]:
df_ofertas_aceptadas.head(2)

### Seleccionamos columnas y hacemos los merge

- De registros nos interesa el id y sacar el json de mortgage data y de ahí:
    * data -> amount, debtRatio
    * property -> zipcode

- De ofertas nos interesa id, mortgage_id, created, del json el nombre del banco
- De ofertas aceptadas, la fecha de aceptación y el bid_id

merge1: solicitudes(id) == ofertas(mortgage_id)
merge2: ofertas_aceptadas(bid_id) = ofertas(id)

In [ ]:
# Solicitudes
amount = []
debtRatio = []
zipcode = []
salary = []
birth = []
work = []

for i in range(df_solicitudes.shape[0]):
    amount.append(json.loads(df_solicitudes.iloc[i,2])['data']['amount'])
    debtRatio.append(json.loads(df_solicitudes.iloc[i,2]).get('data').get('debtRatio'))
    zipcode.append(json.loads(df_solicitudes.iloc[i,2])['property']['zipcode'])
    salary.append(json.loads(df_solicitudes.iloc[i,2])['revenue']['monthly'])
    birth.append(json.loads(df_solicitudes.iloc[i,2])['user']['birthdate'])
    work.append(int(json.loads(df_solicitudes.iloc[i,2])['user']['work']))


In [ ]:
my_dict = {'id_solicitud': df_solicitudes.loc[:, 'id'].copy(),
           'amount': amount.copy(),
           'debtRatio': debtRatio.copy(),
           'zipcode': zipcode.copy(),
           'salary': salary.copy(),
           'birth': birth.copy(),
           'work': work.copy()}

df_solicitudes_clean = pd.DataFrame(my_dict)
df_solicitudes_clean.head(5)

In [ ]:
# Ofertas
df_ofertas_clean = df_ofertas.loc[:,['id', 'mortgage_id', 'created']].copy()
df_ofertas_clean.columns = ['id_oferta', 'mortgage_id', 'date_offer']
df_ofertas_clean.head(2)

In [ ]:
# Ofertas aceptadas
df_ofertas_aceptadas_clean = df_ofertas_aceptadas.loc[:, ['bid_id', 'solicitor_accepted']].copy()
df_ofertas_aceptadas_clean.columns = ['bid_id', 'date_accepted']

In [ ]:
# Merge 1
df_merge1 = pd.merge(left=df_solicitudes_clean, right=df_ofertas_clean, left_on='id_solicitud', right_on='mortgage_id')
df_merge1.head(2)

In [ ]:
# Merge 2
df_merge2 = pd.merge(left=df_merge1, right=df_ofertas_aceptadas_clean, left_on='id_oferta', right_on='bid_id', how='outer')
df_merge2.head(2)

In [ ]:
df = df_merge2.drop(['bid_id', 'mortgage_id'], axis=1).copy()

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
df['amount'] = df['amount'].astype(float).copy()
df['salary'] = pd.to_numeric(df['salary'], errors='coerce')
df['date_offer'] = pd.to_datetime(df['date_offer'], utc=True).copy()
df['date_accepted'] = pd.to_datetime(df['date_accepted'], utc=True, errors='coerce').copy()
df['age'] = (pd.to_datetime('2025-05-25', utc=True)- pd.to_datetime(df['birth'], utc=True, errors='coerce').copy()).dt.days//365
df = df.drop('birth', axis=1)

In [ ]:
df = df.copy()
df.info()

In [ ]:
mask_validos = df['date_offer'] >= '2024-01-01'
df = df.loc[mask_validos, :].copy()

In [ ]:
df.info()

In [ ]:
df['date_accepted']

In [ ]:
(df.groupby('id_solicitud')['date_accepted'].count() == 0).count() #sort_values(ascending=True)

In [ ]:
df.loc[df['id_solicitud'] == '002bd62f-6aa3-46f5-8715-ab17d8d1e57f', :]

In [ ]:
mask_aceptados = df['date_accepted'].isnull() == False
mask_aceptados[mask_aceptados].count()

In [ ]:
# Crear una nueva columna con el valor máximo de 'date_accepted' por 'id_solicitud'
df_aceptados = df[mask_aceptados].copy()
df_aceptados['max_date_accepted'] = df_aceptados.groupby('id_solicitud')['date_accepted'].transform('max')

# Filtrar solo las filas donde 'date_accepted' sea igual a la máxima para ese id
df_aceptados_unicos = df_aceptados[df_aceptados['date_accepted'] == df_aceptados['max_date_accepted']].copy()

In [ ]:
df_solicitudes_unicas = df.drop_duplicates(subset='id_solicitud', keep='first').copy()
df_solicitudes.shape

In [ ]:
df_solicitudes_unicas

### Eliminamos los outliers

In [ ]:
def remove_outliers_iqr(df, column, iqr_multiplier=1.5):
    """
    Removes outliers from a specified column in a DataFrame using the IQR method.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the column to remove outliers from.
        iqr_multiplier (float): The multiplier for the IQR to define the outlier fences.
                                  Commonly 1.5 for "mild" outliers, 3.0 for "extreme" outliers.

    Returns:
        pd.DataFrame: A new DataFrame with outliers removed.
    """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame.")

    # Calculate Q1, Q3, and IQR, ignoring NaN values
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier fences
    lower_bound = Q1 - iqr_multiplier * IQR
    upper_bound = Q3 + iqr_multiplier * IQR

    # Filter out outliers. Keep rows where the value is within the bounds OR is NaN.
    # We explicitly include NaN values so they are not treated as outliers and are preserved
    # if you want to handle them separately later.
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound) | (df[column].isna())]

    # Optional: Print how many outliers were removed
    num_outliers = len(df) - len(filtered_df)
    print(f"Removed {num_outliers} outliers from '{column}' using IQR method (multiplier={iqr_multiplier}).")

    return filtered_df

columns = ['amount', 'salary', 'debtRatio']

for column in columns:
    df_solicitudes_unicas = remove_outliers_iqr(df_solicitudes_unicas, column)
    df_aceptados = remove_outliers_iqr(df_aceptados, column)
    df_aceptados_unicos = remove_outliers_iqr(df_aceptados_unicos, column)

### Empezamos el análisis

In [ ]:
# Primero qué tipo de cliente se interesa por nuestra empresa:
fig, ax = plt.subplots(5, 1, figsize=(8,8))
fig.suptitle('Solicitudes')
sns.histplot(data=df_solicitudes_unicas, x='amount', ax=ax[0])
sns.histplot(data=df_solicitudes_unicas, x='salary', ax=ax[1])
sns.histplot(data=df_solicitudes_unicas, x='debtRatio', ax=ax[2])
sns.histplot(data=df_solicitudes_unicas, x='age', ax=ax[3])
sns.countplot(data=df_solicitudes_unicas, x='work', ax=ax[4])

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,4))
fig.suptitle('Edad')
sns.histplot(data=df_solicitudes_unicas, x='age', ax=ax, color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax.set_xlabel('Edad [años]')
ax.set_ylabel('Frecuencia')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,4))
fig.suptitle('Trabajo')
sns.countplot(data=df_solicitudes_unicas, x='work', ax=ax, color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax.set_xlabel('Sector de trabajo')
ax.set_ylabel('Frecuencia')

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12,8))
#fig.suptitle('Trabajo')
sns.histplot(data=df_solicitudes_unicas, x='amount', ax=ax[0,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax[0,0].set_xlabel('Hipoteca [€]')
ax[0,0].set_ylabel('Frecuencia')
ax[0,0].set_title('Hipoteca', fontsize=25)

sns.histplot(data=df_solicitudes_unicas, x='salary', ax=ax[0,1], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax[0,1].set_xlabel('Sueldo [€]')
ax[0,1].set_ylabel('Frecuencia')
ax[0,1].set_title('Sueldo', fontsize=25)

sns.histplot(data=df_solicitudes_unicas, x='age', ax=ax[1,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax[1,0].set_xlabel('Edad [años]')
ax[1,0].set_ylabel('Frecuencia')
ax[1,0].set_title('Edad', fontsize=25)

sns.histplot(data=df_solicitudes_unicas, x='debtRatio', ax=ax[1,1], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax[1,1].set_xlabel('Ratio de deuda')
ax[1,1].set_ylabel('Frecuencia')
ax[1,1].set_title('Ratio de deuda', fontsize=25)


plt.tight_layout() 

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12,8))
#fig.suptitle('Trabajo')
sns.histplot(data=df_solicitudes_unicas, x='age', ax=ax[0,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax[0,0].set_xlabel('Edad [años]')
ax[0,0].set_ylabel('Frecuencia')
ax[0,0].set_title('Solicitud', fontsize=25)

sns.histplot(data=df_aceptados_unicos, x='age', ax=ax[0,1], color=COLORES_COLIBID['secondary'], edgecolor='black')
ax[0,1].set_xlabel('Edad [años] [€]')
ax[0,1].set_ylabel('Frecuencia')
ax[0,1].set_title('Oferta aceptada', fontsize=25)

sns.histplot(data=df_solicitudes_unicas, x='debtRatio', ax=ax[1,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax[1,0].set_xlabel('Ratio deuda')
ax[1,0].set_ylabel('Frecuencia')
#ax[1,0].set_title('Edad', fontsize=25)

sns.histplot(data=df_aceptados_unicos, x='debtRatio', ax=ax[1,1], color=COLORES_COLIBID['secondary'], edgecolor='black')
ax[1,1].set_xlabel('Ratio deuda')
ax[1,1].set_ylabel('Frecuencia')
#ax[1,1].set_title('Ratio de deuda', fontsize=25)

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(4,6))
#fig.suptitle('Trabajo')
sns.countplot(data=df_solicitudes_unicas, x='work', ax=ax[0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax[0].set_xlabel('Sector de trabajo')
ax[0].set_ylabel('Frecuencia')
ax[0].set_title('Solicitud', fontsize=25)

sns.countplot(data=df_aceptados_unicos, x='work', ax=ax[1], color=COLORES_COLIBID['secondary'], edgecolor='black')
ax[1].set_xlabel('Sector de trabajo')
ax[1].set_ylabel('Frecuencia')
ax[1].set_title('Oferta aceptada', fontsize=25)

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(6, 2, figsize=(12, 12))
# fig.suptitle('Solicitudes')
sns.histplot(data=df_solicitudes_unicas, x='amount', ax=ax[0,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
sns.histplot(data=df_solicitudes_unicas, x='salary', ax=ax[1,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
sns.histplot(data=df_solicitudes_unicas, x='debtRatio', ax=ax[2,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
sns.histplot(data=df_solicitudes_unicas, x='age', ax=ax[3,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
sns.countplot(data=df_solicitudes_unicas, x='work', ax=ax[4,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
sns.histplot(data=df_solicitudes_unicas, x='date_offer', ax=ax[5,0], color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')

sns.histplot(data=df_aceptados_unicos, x='amount', ax=ax[0,1], color=COLORES_COLIBID['secondary_orange'], edgecolor='black')
sns.histplot(data=df_aceptados_unicos, x='salary', ax=ax[1,1], color=COLORES_COLIBID['secondary_orange'], edgecolor='black')
sns.histplot(data=df_aceptados_unicos, x='debtRatio', ax=ax[2,1], color=COLORES_COLIBID['secondary_orange'], edgecolor='black')
sns.histplot(data=df_aceptados_unicos, x='age', ax=ax[3,1], color=COLORES_COLIBID['secondary_orange'], edgecolor='black')
sns.countplot(data=df_aceptados_unicos, x='work', ax=ax[4,1], color=COLORES_COLIBID['secondary_orange'], edgecolor='black')
sns.histplot(data=df_aceptados_unicos, x='date_offer', ax=ax[5,1], color=COLORES_COLIBID['secondary_orange'], edgecolor='black')

plt.tight_layout()
plt.savefig('PrimeraComparacion.png')
plt.show()


for column in columns:
    print(f"""{column}: Solicitud: {df_solicitudes_unicas[column].median()}, aceptada: {df_aceptados_unicos[column].median()}, std_dev: {df_solicitudes_unicas[column].std() - df_aceptados_unicos[column].std()}""")

In [ ]:
# Asume que df_solicitudes_unicas y df_aceptados_unicos ya están definidos
columns = ['amount', 'salary', 'debtRatio', 'age']

for column in columns:
    print(f"\n--- Análisis de la Columna: {column} ---")

    # Calculamos las métricas para cada grupo
    median_solicitudes = df_solicitudes_unicas[column].median()
    median_aceptados = df_aceptados_unicos[column].median()

    mean_solicitudes = df_solicitudes_unicas[column].mean()
    mean_aceptados = df_aceptados_unicos[column].mean()

    std_solicitudes = df_solicitudes_unicas[column].std()
    std_aceptados = df_aceptados_unicos[column].std()

    q1_solicitudes = df_solicitudes_unicas[column].quantile(0.25)
    q1_aceptados = df_aceptados_unicos[column].quantile(0.25)

    q3_solicitudes = df_solicitudes_unicas[column].quantile(0.75)
    q3_aceptados = df_aceptados_unicos[column].quantile(0.75)

    # Coeficiente de Variación (CV), usa la media como denominador
    cv_solicitudes = (std_solicitudes / mean_solicitudes) * 100 if mean_solicitudes != 0 else np.nan
    cv_aceptados = (std_aceptados / mean_aceptados) * 100 if mean_aceptados != 0 else np.nan

    # Imprimimos los resultados para una fácil comparación
    print(f"  Métrica        | Solicitudes         | Aceptados           | Diferencia (Aceptados - Solicitudes)")
    print(f"-----------------|---------------------|---------------------|--------------------------------------")
    print(f"  Mediana        | {median_solicitudes:<19.2f} | {median_aceptados:<19.2f} | {median_aceptados - median_solicitudes:<25.2f}")
    print(f"  Media          | {mean_solicitudes:<19.2f} | {mean_aceptados:<19.2f} | {mean_aceptados - mean_solicitudes:<25.2f}")
    print(f"  Std Dev        | {std_solicitudes:<19.2f} | {std_aceptados:<19.2f} | {std_aceptados - std_solicitudes:<25.2f}")
    print(f"  Q1 (25%)       | {q1_solicitudes:<19.2f} | {q1_aceptados:<19.2f} | {q1_aceptados - q1_solicitudes:<25.2f}")
    print(f"  Q3 (75%)       | {q3_solicitudes:<19.2f} | {q3_aceptados:<19.2f} | {q3_aceptados - q3_solicitudes:<25.2f}")
    print(f"  CV (%)         | {cv_solicitudes:<19.2f} | {cv_aceptados:<19.2f} | {cv_aceptados - cv_solicitudes:<25.2f}")

In [ ]:
import matplotlib.dates as mdates
ax = sns.histplot(df_solicitudes_unicas['date_offer'], 
            color=COLORES_COLIBID['primary_blue_green'],
            edgecolor='black')

ax.set_ylabel('Frecuencia')
ax.set_xlabel('Fecha')

# Formatear el eje X
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))  # cada 2 meses
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.tick_params(axis='x', rotation=45)  # rotar para mejor lectura

plt.tight_layout()
plt.show()

In [ ]:
df_aceptados_unicos['tiempo_aceptacion'] = (df_aceptados_unicos['date_accepted'] - df_aceptados_unicos['date_offer']).dt.total_seconds() / (60 * 60 * 24)
df_aceptados_unicos.shape

# Now plot it
ax = sns.histplot(
    df_aceptados_unicos['tiempo_aceptacion'], 
    color=COLORES_COLIBID['primary_blue_green'],
    edgecolor='black',
    bins=30  # optional: set number of bins
)

ax.set_xlabel('Tiempo de aceptación (días)')
ax.set_ylabel('Frecuencia')
plt.tight_layout()
plt.show()

In [ ]:
ax = sns.barplot(df_solicitudes_unicas['date_offer'].dt.day_name(), 
            order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            errorbar=None,
            color=COLORES_COLIBID['primary_blue_green'],
            edgecolor='black')
ax.set_ylabel('Día de la semana')

# Set Spanish labels safely
spanish_labels = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
ax.set_yticks(range(len(spanish_labels)))  # set fixed tick positions
ax.set_yticklabels(spanish_labels)    

plt.tight_layout()
plt.show()

#### Intentamos localizar las hipotecas en un mapa

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [ ]:
df_locs = pd.DataFrame(df_solicitudes_unicas.groupby('zipcode')['id_oferta'].count().sort_values(ascending=False)).reset_index()

In [ ]:
df_locs.columns=['zipcode', 'counts']

In [ ]:
df_excel = pd.read_excel('data/Listado-de-CP.xlsx')
df_excel.head()

In [ ]:
df_excel.loc[df_excel['codigo_postal'] == 46910, :]
df_excel.info()
df_excel['codigo_postal'] = df_excel['codigo_postal'].astype(str)

In [ ]:
df_locs2 = pd.merge(left=df_locs, right=df_excel, left_on='zipcode', right_on='codigo_postal', how='inner')
df_locs = df_locs2.drop(['codigo_postal', 'municipio_id', 'codigo_unidad_poblacional', 'entidad_singular_nombre'], axis=1)
df_locs


In [ ]:
fig, ax = plt.subplots(1,1)
sns.barplot(data=df_locs.iloc[:20, :], x='zipcode', y='counts', errorbar=None, color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.tight_layout()

In [ ]:
df_locs_plot = pd.DataFrame(df_locs.groupby('nucleo_nombre')['zipcode'].count().sort_values(ascending=False)).reset_index()
df_locs_plot.columns=['nombre_nucleo', 'counts']


In [ ]:
fig, ax = plt.subplots(1,1, figsize = (15, 7))
sns.barplot(data=df_locs_plot.iloc[1:11, :], x='nombre_nucleo', y='counts', errorbar=None, color=COLORES_COLIBID['primary_blue_green'], edgecolor='black')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=15)
ax.set_xlabel('Localidad')
ax.set_ylabel('Frecuencia')
ax.set_title('Top 10 localidades de contratación de hipotecas', fontsize=20)
plt.tight_layout()

In [ ]:
fig_scatter = px.scatter_mapbox(df_locs,
                                lat='Latitud',
                                lon='Longitud',
                                color='counts',
                                color_continuous_scale=px.colors.sequential.YlOrRd, # Escala de colores (amarillo a rojo)
                                size_max=30, # Tamaño máximo de los marcadores para evitar que sean demasiado grandes
                                zoom=5.5, # Nivel de zoom inicial (ajusta según tu región)
                                # Centra el mapa en la media de tus puntos para una vista general
                                center={"lat": df_locs['Latitud'].mean(), "lon": df_locs['Longitud'].mean()},
                                mapbox_style="open-street-map", # Estilo de mapa base (no requiere clave de Mapbox)
                                title=f"Puntos Calientes de Actividad Hipotecaria por Código Postal (España)"
                               )

# Ajustes adicionales para el layout del mapa
fig_scatter.update_layout(
    margin={"r":0,"t":50,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title='cuentas',
        tickformat="," # Formato para los números en la leyenda de color
    )
)

fig_scatter.show()

In [ ]:
fig_density = px.density_mapbox(df_locs,
                                lat="Latitud",
                                lon="Longitud",
                                z='counts', # Variable que define la intensidad del calor
                                radius=20, # Ajusta el radio de los puntos de calor (mayor radio = calor más extendido)
                                color_continuous_scale=px.colors.sequential.Teal,#['#E5F5F1', '#80BEAB', '#03897D'],#px.colors.sequential.Hot, # Otra buena escala de colores para calor
                                zoom=5.5,
                                center={"lat": df_locs['Latitud'].mean(), "lon": df_locs['Longitud'].mean()},
                                mapbox_style="open-street-map",
                                title=f"Mapa de Densidad de Actividad Hipotecaria por Código Postal (España)"
                               )

fig_density.update_layout(
    margin={"r":0,"t":50,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title='Número de hipotecas',
        tickformat="," # Formato para los números en la leyenda de color
    ))
fig_density.show()